In [1]:
import os
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd
import math
import platform

In [2]:
# rootdir = os.path.dirname(os.path.abspath(__file__))

# landcover_directory = r"\\akif.internal\public\z_resources\im-wb\landcove_layers"
landcover_directory = r"Z:\z_resources\im-wb\landcover_layers"
landcover_classes_csv = "./tmp/landcover_classes.csv" #this works

In [3]:
def get_raster_data(path):
    file_list = []
    for file in os.listdir(path):
        # Iterate over all the files in the specified directory.
        if ".tif" in file:
            # Process the file if it has a .tif format.
            if platform.system() is "Windows":
                address = os.path.join(path, file).replace("/","\\")
            else:
                address = os.path.join(path, file).replace("\\","/")
                #build the path according the OS running the script

            if address not in file_list:
                # Add the file address to the list if it had not been added before.
                file_list.append(address)
        else:
            pass
    
    return file_list

landcover_list = get_raster_data(landcover_directory)



<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\ruben.crespo\AppData\Local\Temp\ipykernel_17684\3629118024.py:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if platform.system() is "Windows":


In [4]:
clc_df = pd.read_csv(landcover_classes_csv) #this works
for raster_file in landcover_list[0:1]:
    #create a folder for the outputs
    """Here we create and go the folder which will contain the landcover reclass"""
    data_name = raster_file.replace('.tif','')
    output_dir = os.path.join(landcover_directory, data_name + "_reclass")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    

In [ ]:
def split_the_raster(raster_image, width, height):

    tilesize = 1000

    for i in range(0, width, tilesize): # Tilesize marks from where to where in width.
        for j in range(0, height, tilesize):
            # This is for the edge parts, so we don't get nodata from the borders.
            w0 = i # Start of the array.
            w_plus = min(i+tilesize, width) - i # Addition value.
            w1 = w0 + w_plus # End of the array.
            h0 = j
            h_plus = min(j+tilesize, height) - j
            h1 = h0 + h_plus


    return accumulated_agg_value

In [7]:
for row_index, row in clc_df.loc[0:1].iterrows():  
        # Load the raster file.
        landcover_file = rasterio.open(raster_file)
        landcover_array = landcover_file.read() #[z,x,y]
        height = landcover_array.shape[1]
        width  = landcover_array.shape[2]
        #create a copy
        # landcover_reclass = landcover_file.copy()
        
        # # do a reclass of the file
        # landcover_reclass[np.where(landcover_reclass = row['VALUE'] )] = 1
        # landcover_reclass[np.where(landcover_reclass != row['VALUE'] )] = 0
        
     

In [ ]:
def reclassifier(landcover_layer):
    landcover_layer[np.where(landcover_layer = row['VALUE'] )] = 1
    landcover_layer[np.where(landcover_layer != row['VALUE'] )] = 0
    return landcover_reclass

In [ ]:
#save the result
with rasterio.open(os.path.join(output_dir, data_name, '_', row['VALUE'],'.tif').replace("/","\\"), 'w', 
                    driver = landcover_file.driver, 
                    height = landcover_file.height, 
                    width = landcover_file.width, 
                    count = landcover_file.count, 
                    dtype = landcover_file.dtype, 
                    crs = landcover_file.crs, 
                    transform = landcover_file.transform, 
                    compress='deflate') as dst:
    
    dst.write(landcover_reclass)